Make sure you have Python 3.8.

In [ ]:
import sys
! {sys.executable} --version

**Imports**

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import os
import pandas  # used to load metadata table
import numpy as np

**Import midtools methods**

In [ ]:
import midtools
from midtools.scheduler import Scheduler
from midtools.scheduler import start_midtools

## Setting Paths

Specify the output directory for the results (`BASE_DIR`) and for the SLURM logs (`JOB_DIR`). The later will be used for monitoring the progress of the jobs.

In [ ]:
JOB_DIR = "./jobs"
BASE_DIR = "./analyzed_runs/"

Midtools is shipped with a mask but you probably want to make your own at a later stage. The mask needs to by stored as an `npy` file and must have the shape (16,512,128).

In [ ]:
maskfile = midtools.__path__[0]+'/setup_config/darkmask_r0283_00.npy'
print(f"Mask exists: {os.path.exists(maskfile)}.")
print(f"Maskfile is: {maskfile}")
print(f"Maskshape is: {np.load(maskfile).shape}")

## Setupfiles

Setupfiles are used to set most of the parameters. Next to each result-file the setupfile is stored that was used to generate the results.

**Change the maskfile!**: to the maskfile from the last cell.

In [ ]:
%%writefile base_setupfile.yml
mask: /gpfs/exfel/data/user/reiserm/projects/proteins/mid-p2458/midtools/midtools/setup_config/darkmask_r0283_00.npy
photon_energy: 9
pixel_size: 200
quadrant_positions:
  dx: -4.0
  dy: 14.0
  q1:
  - -540.0
  - 657.0
  q2:
  - -610.0
  - -34.0
  q3:
  - 530.0
  - -230.0
  q4:
  - 582.0
  - 470.0
sample_detector: 8
saxs_opt:
    integrate: 1d
    nqbins: 256
#     nphibins: 128
xpcs_opt:
  norm: symmetric
  q_range:
    nsteps: 4
    q_first: 0.1
    q_last: 0.4
    scale: linear
#   phi_range:
#     phi_width: 45
#     phi_first: 0
#     phi_last: 180
#     scale: linear
diagnostics_opt:
    xgm_threshold: 200
        
slurm_opt:
    njobs: 8
    nprocs: 4
    partition: upex

Let's have a look at the created setupfile.

In [ ]:
!cat base_setupfile.yml

The metadata of our MID beamtime in fall 2020 is available, but usually you need to figure out all the parameters yourself or ask your local contact. As an example, we analyze run 280 of experiment 2718, which is nanoparticles in water. 

In [ ]:
# example silica nanoparticles
run_number = 280
proposal = 2718

# Ig-PEG 
# run_number = 300  #340 for different rates
# proposal = 2718

# Ig-PEG 
# run_number = 240  #340 for different rates
# proposal = 2718

# Ig-PEG 
# run_number = 260  #340 for different rates
# proposal = 2718


# this only works when you have created a metadata datatable
meta = pandas.read_pickle(midtools.__path__[0]+'/setup_config/metadata.pkl')
meta.loc[(proposal,run_number)]

`starts_midtools` submits a job to run the analysis. The keyword argument `test` can be set to see the batch file without actually submitting the job. The parameters in the next cell have been set for you. Check the output and then change `TEST=False` to submit the job.

**Note:** The data direcotry (datdir) in the metadata table is not correct, but the datdir in the next cell is.

In [ ]:
TEST = False

start_midtools(
          proposal,
          run_number,
          setupfile='base_setupfile.yml',
          pulses_per_train=200,
          datdir='/gpfs/exfel/exp/MID/202002/p002718/proc/',
          first_train=0,
          last_train=1000,
          first_cell=130,
          pulse_step=1,
          test=TEST,
          jobdir=JOB_DIR,
          base_dir=BASE_DIR,
          saxs=True,
          xpcs=True,
          frames=True, 
)

## Monitor Jobs

Use the following set of widgets to monitor your analysis jobs. Start by clicking on `Update Job Table`. `list jobs` will list all your SLURM jobs.

In [ ]:
scheduler = Scheduler(JOB_DIR)  
scheduler.gui()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
img = np.load('./analyzed_runs/p002718/rois.npy')
plt.figure()
plt.imshow(img, cmap='tab20')

## Open HDF File

Check the contents of the resulting HDF5 file.

In [ ]:
import h5py

In [ ]:
def printf(x):
    if isinstance(f[x], h5py._hl.dataset.Dataset):
        print(f[x].name, f[x].shape)

In [ ]:
filename = './analyzed_runs/p002718/r0280-analysis_000.h5'
with h5py.File(filename, 'r') as f:
    f.visit(printf)